# <font color='red'>1.0 IMPORT</font>

In [1]:
# ENVIRONMENT

# !pip install mysql-connector-python
# !pip install pymysql
# !pip install sqlalchemy

import pandas as pd
import mysql.connector
import numpy as np
from sqlalchemy import create_engine

# pd.options.display.max_rows = 2000
# pd.options.display.width = 120
# pd.options.display.max_colwidth = 100


# <font color='red'>2.0 EXTRACT</font>

In [2]:
def LeFontes(strAut):
    sql_form = (
    "SELECT `atributo`, `valor`, `protocolo` "
    "FROM form "
    "WHERE `atributo` IN ('state', 'city', 'neighborhood', 'zipcode', 'street')"
    )
    sql_tasks = (
    "SELECT `Protocolo`, `Entidade`, `Serviço`, `Usuário`, `Grupo`, `Data e Hora de conclusão`, "
    "`Data e Hora de criação`, `Ação`, `Encaminhado para`, `Processo encerrado`, `Processo cancelado`, "
    "`Motivo de cancelamento`, `Status externo`, `Categoria`, `Grupo responsável`, `Prazo (em segundos)` "
    "FROM tasks"
    )
    sql_sla = (
    "SELECT * "
    "FROM sla"
    )
    sql_rating = (
    "SELECT * "
    "FROM rating"
    )
    
    try:
        connection = mysql.connector.connect(host     = strAut['My_host'], 
                                             database = strAut['My_db'], 
                                             user     = strAut['My_user'], 
                                             password = strAut['My_pw'])

        df_int_tasks =   pd.read_sql(sql_tasks,  con=connection)
        df_int_sla =     pd.read_sql(sql_sla,    con=connection)
        df_int_form =    pd.read_sql(sql_form,   con=connection)
        df_int_rating =  pd.read_sql(sql_rating, con=connection)

    except mysql.connector.Error as error:
        print("Failed to read record from MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            connection.close()
            print(f'tasks:{df_int_tasks.shape[0]} registros lidos em {df_int_tasks.shape[1]} colunas')
            print(f'sla: {df_int_sla.shape[0]} registros lidos em {df_int_sla.shape[1]} colunas')
            print(f'form: {df_int_form.shape[0]} registros lidos em {df_int_form.shape[1]} colunas')
            print(f'rating: {df_int_rating.shape[0]} registros lidos em {df_int_rating.shape[1]} colunas')
            print("MySQL connection is closed")
            
    return df_int_tasks, df_int_sla, df_int_form, df_int_rating

# <font color='red'>3.0 TRANSFORM</font>

# <font color='blue'>trata os datasets</font>

## <font color='black'>trata o dataset sla</font>

In [3]:
def trSLA(df):
    MINUTOS_PARA_DIAS = 1440 # quantidade de minutos em um dia

    # preenche com null as colunas que contém 'null' como tipo string
    df.loc[(df['limiteMinimo'] == 'null'), 'limiteMinimo'] = np.nan
    df.loc[(df['limiteMaximo'] == 'null'), 'limiteMaximo'] = np.nan

    # transforma o tipo de coluna para float
    df['limiteMinimo'] = df['limiteMinimo'].astype(float)
    df['limiteMaximo'] = df['limiteMaximo'].astype(float)

    # transforma a unidade de medida de minuto para dias
    df['limiteMinimo'] = df['limiteMinimo'] / MINUTOS_PARA_DIAS
    df['limiteMaximo'] = df['limiteMaximo'] / MINUTOS_PARA_DIAS

    # preenche com valores extremos os limites máximos e limites mínimos
    df.loc[(df['limiteMinimo'].isna()), 'limiteMinimo'] = -9999999.9
    df.loc[(df['limiteMaximo'].isna()), 'limiteMaximo'] = 9999999.9

    # cria mais uma coluna de status para pivotar limite mínimo e limite máxio
    df['status2'] = df['status']
    df.rename(columns={'status': 'StatusLmin', 'status2': 'StatusLmax'}, inplace = True)
    
    # faz pivot da coluna StatusLmin
    idx = ['entityCode', 'service', 'StatusLmax', 'limiteMaximo']
    df = df.pivot(columns = 'StatusLmin', values = 'limiteMinimo', index=idx).reset_index()
    df.columns.name = None
    dic_renome = {'Dentro do prazo' : 'Dentro do Prazo LMin', 
                  'Fora do prazo' : 'Fora do Prazo LMin',
                  'Perto do prazo' : 'Perto do Prazo LMin'}
    df.rename(columns=dic_renome, inplace = True)

    # faz pivot da coluna StatusLmax
    idx = ['entityCode', 'service', 'Dentro do Prazo LMin', 'Fora do Prazo LMin', 'Perto do Prazo LMin']
    df = df.pivot(columns = 'StatusLmax', values = 'limiteMaximo', index=idx).reset_index()
    df.columns.name = None
    dic_renome = {'Dentro do prazo' : 'Dentro do Prazo LMax', 
                  'Fora do prazo' : 'Fora do Prazo LMax',
                  'Perto do prazo' : 'Perto do Prazo LMax'}
    df.rename(columns=dic_renome, inplace = True)

    # agrupa por entidade e serviço
    df = df.groupby('service').sum().reset_index()

    print('df_sla transformado')
    
    return df

## <font color='black'>trata o dataset form</font>

In [4]:
def trForm(df):
    
    # faz pivot da coluna atributo
    df = df.pivot(columns='atributo', values='valor', index='protocolo').reset_index()
    df.columns.name = None

    # renomeia colunas
    dic_renome = {'protocolo': 'Solicitacao', 
                  'zipcode': 'CEP', 
                  'street': 'Endereco', 
                  'neighborhood': 'Bairro', 
                  'city': 'Cidade', 
                  'state': 'UF'}
    df.rename(columns=dic_renome, inplace = True)
    
    # substitui a coluna Endereco por branco quando '-' ou null
    df.loc[(df['Endereco'] == '-') | (df['Endereco'].isnull()), ['Endereco']] = ''
    df.loc[(df['Bairro'] == '-') | (df['Bairro'].isnull()), ['Bairro']] = ''

    # cria a coluna Endereco Completo
    df['EnderecoCompleto'] = df['Cidade'] + ', ' + df['UF'] + ', Brasil'
    
    print('df_form transformado')

    return df

## <font color='black'>trata o dataset rating</font>

In [5]:
def trRating(df):

    # renomeia as colunas
    cols = ['Solicitacao', 'NotaAvaliacao', 'MotivoAvaliacao', 'DataHoraAvaliacao']
    df.columns = cols

    # separa e formata as colunas de datas e horas 
    df['DataAvaliacao'] = pd.to_datetime(df['DataHoraAvaliacao']).dt.date
    df['HoraAvaliacao'] = pd.to_datetime(df['DataHoraAvaliacao']).dt.time

    # deleta a coluna que contém data e hora
    df.drop(['DataHoraAvaliacao'], axis=1, inplace=True)

    # preenche colunas de linhas vazias
    df.loc[df['MotivoAvaliacao'] == '', 'MotivoAvaliacao'] = '<motivo vazio>'
    
    print('df_rating transformado')
    
    return df

## <font color='black'>trata o dataset tasks</font>

In [6]:
def trTasks(df):

    SEM_STATUS = '<sem status inicial>'
    
    # renomeia as colunas
    lst_colunas_tasks = ['Protocolo', 'Entidade', 'Servico', 'Usuarios','Grupo', 'DataHora_Conclusao',
                          'DataHora_Criacao', 'Acao', 'EncaminhadoPara', 'ProcessoEncerrado', 'ProcessoCancelado',
                          'MotivoCancelamento', 'StatusExterno', 'Categoria', 'GrupoResponsavel','Prazo']
    df.columns = lst_colunas_tasks

    # rotina para preencher status externo vazio
    # pega sempre o anterior e se o primeiro status estiver vazio coloca "sem status inicial"
    if df.loc[0, 'StatusExterno'] == '':
        df.loc[0, 'StatusExterno'] = SEM_STATUS
    for i in range(1, df.shape[0]):
        if df.loc[i, 'StatusExterno'] == '':    
            if df.loc[i, 'Protocolo'] == df.loc[i - 1, 'Protocolo']:
                df.loc[i, 'StatusExterno'] = df.loc[i - 1, 'StatusExterno']
            else:
                df.loc[i, 'StatusExterno'] = SEM_STATUS

    if df.loc[0, 'StatusExterno'] == '':
        df.loc[0, 'StatusExterno'] = '<sem status inicial>'
    for i in range(1, df.shape[0]):
        if df.loc[i, 'StatusExterno'] == '':    
            if df.loc[i, 'Protocolo'] == df.loc[i - 1, 'Protocolo']:
                df.loc[i, 'StatusExterno'] = df.loc[i - 1, 'StatusExterno']
            else:
                df.loc[i, 'StatusExterno'] = '<sem status inicial>'
    
    print('df_tasks transformado')

    return df

# <font color='blue'>monta tabelas dimensões</font>

## DIM acoes

In [7]:
def Mt_dim_Acoes(df):

    # trasnforma a coluna Acao em uma coluna do tipo "category'
    df['tmpAcao'] = df['Acao'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Ação
    df['FK_dim_Acoes'] = df['tmpAcao'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Acoes', 'Acao']].drop_duplicates()
    dfx.rename(columns={'FK_dim_Acao': 'PK_dim_Acao'}, inplace = True)
    
    # preenche colunas com linhas vazias
    dfx.loc[dfx['Acao'] == '', ['Acao']] = '<sem ação determinada>'
    
    # exclui a coluna Acoes de df_tasks que será a futura tabela fato
    df.drop(['Acao', 'tmpAcao'], axis=1, inplace=True)
    
    print('dim_Acoes montada')

    return dfx

## DIM categoria servicos

In [8]:
def Mt_dim_CategoriaServico(df):

    # trasnforma a coluna Categoria em uma coluna do tipo "category'
    df['tmpCategoria'] = df['Categoria'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão CategoriasServicos
    df['FK_dim_CategoriasServicos'] = df['tmpCategoria'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_CategoriasServicos', 'Categoria']].drop_duplicates()
    dfx.rename(columns={'FK_dim_CategoriasServicos': 'PK_dim_CategoriasServicos'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Categoria'] == '', ['Categoria']] = '<categoria indefinida>'
    
    # exclui a coluna Categoria de df_tasks que será a futura tabela fato
    df.drop(['Categoria', 'tmpCategoria'], axis=1, inplace=True)
    
    print('dim_Categoria montada')

    return dfx

## DIM encaminhamento

In [9]:
def Mt_dim_Encaminhamento(df):

    # trasnforma a coluna EncaminhadoPara em uma coluna do tipo "category'
    df['tmpEncaminhadoPara'] = df['EncaminhadoPara'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Encaminhamento
    df['FK_dim_Encaminhamento'] = df['tmpEncaminhadoPara'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Encaminhamento', 'EncaminhadoPara']].drop_duplicates()
    dfx.rename(columns={'FK_dim_Encaminhamento': 'PK_dim_Encaminhamento'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['EncaminhadoPara'] == '', ['EncaminhadoPara']] = '<sem encaminhamento>'

    # exclui a coluna Entidade de df_tasks que será a futura tabela fato
    df.drop(['EncaminhadoPara', 'tmpEncaminhadoPara'], axis=1, inplace=True)

    print('dim_Encaminhamento montada')

    return dfx

## DIM entidade

In [10]:
def Mt_dim_Entidade(df):


    # trasnforma a coluna Entidade em uma coluna do tipo "category'
    df['tmpEntidade'] = df['Entidade'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Entidades
    df['FK_dim_Entidades'] = df['tmpEntidade'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Entidades', 'Entidade']].drop_duplicates()
    dfx['Entidade'] = dfx['Entidade'].str.upper()
    dfx.rename(columns={'FK_dim_Entidades': 'PK_dim_Entidades'}, inplace = True)

    # exclui a coluna Entidade de df_tasks que será a futura tabela fato
    df.drop(['Entidade', 'tmpEntidade'], axis=1, inplace=True)

    print('dim_Entidade montada')

    return dfx

## DIM grupo responsavel

In [11]:
def Mt_dim_GrupoResponsavel(df):
    
    # trasnforma a coluna GrupoResponsavel em uma coluna do tipo "category'
    df['tmpGrupoResponsavel'] = df['GrupoResponsavel'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão GrupoResponsavel
    df['FK_dim_GrupoResponsavel'] = df['tmpGrupoResponsavel'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_GrupoResponsavel', 'GrupoResponsavel']].drop_duplicates()
    dfx.rename(columns={'FK_dim_GrupoResponsavel': 'PK_dim_GrupoResponsavel'}, inplace = True)

    # exclui a coluna GrupoResponsavel de df_tasks que será a futura tabela fato
    df.drop(['GrupoResponsavel', 'tmpGrupoResponsavel'], axis=1, inplace=True)
    
    print('dim_GrupoResponsavel montada')
    
    return dfx

## DIM grupo usuarios

In [12]:
def Mt_dim_GruposUsuarios(df):

    # trasnforma a coluna Grupo em uma coluna do tipo "category'
    df['tmpGrupo'] = df['Grupo'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Grupo
    df['FK_dim_GruposUsuarios'] = df['tmpGrupo'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_GruposUsuarios', 'Grupo']].drop_duplicates()
    dfx.rename(columns={'FK_dim_GruposUsuarios': 'PK_dim_GruposUsuarios'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Grupo'] == '', ['Grupo']] = '<grupo usuário não definido>'

    # exclui a coluna Grupo de df_tasks que será a futura tabela fato
    df.drop(['Grupo', 'tmpGrupo'], axis=1, inplace=True)
    
    print('dim_GruposUsuarios montada')
    
    return dfx

## DIM motivos cancelamento

In [13]:
def Mt_MotivosCanc(df):


    # trasnforma a coluna MotivoCancelamento em uma coluna do tipo "category'
    df['tmpMotivoCancelamento'] = df['MotivoCancelamento'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão MotivoCanc
    df['FK_dim_MotivosCanc'] = df['tmpMotivoCancelamento'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_MotivosCanc', 'MotivoCancelamento']].drop_duplicates()
    dfx.rename(columns={'FK_dim_MotivosCanc': 'PK_dim_MotivosCanc'}, inplace = True)


    # preenche colunas com linhas vazias
    dfx.loc[dfx['MotivoCancelamento'] == '', ['MotivoCancelamento']] = '<sem motivo de cancelamento>'

    # exclui a coluna MotivoCancelamento de df_tasks que será a futura tabela fato
    df.drop(['MotivoCancelamento', 'tmpMotivoCancelamento'], axis=1, inplace=True)

    print('dim_MotivosCanc montada')
    
    return dfx

## DIM servico

In [14]:
def Mt_Servico(df, dff_sla):

    # trasnforma a coluna Servico em uma coluna do tipo "category'
    df['tmpServico'] = df['Servico'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Servico
    df['FK_dim_Servicos'] = df['tmpServico'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Servicos', 'Servico']].drop_duplicates()

    # exclui a coluna Servico de df_tasks que será a futura tabela fato
    df.drop(['Servico', 'tmpServico'], axis=1, inplace=True)

    # defive os nomes das colunas de SLA
    cols = {'FK_dim_Servicos': 'PK_dim_Servicos',
            'Dentro do Prazo LMin': 'sla_VD_Lmin',
            'Perto do Prazo LMin': 'sla_AM_Lmin',
            'Fora do Prazo LMin': 'sla_VM_Lmin',
            'Dentro do Prazo LMax': 'sla_VD_Lmax',
            'Perto do Prazo LMax': 'sla_AM_LMax',
            'Fora do Prazo LMax': 'sla_VM_LMax',
           }
    # faz um merge da dimensão dim_Servicos com a tabela de SLAs
    dfx = dfx.merge(dff_sla, left_on='Servico', right_on='service', how='left')
    dfx = dfx.drop('service', axis=1)
    dfx.rename(columns=cols, inplace = True)
    
    print('dim_Servico montada')
    
    return dfx

## DIM status externo

In [15]:
def Mt_StatusExt(df):

    # trasnforma a coluna StatusExterno em uma coluna do tipo "category'
    df['tmpStatusExterno'] = df['StatusExterno'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Grupo
    df['FK_dim_StatusExt'] = df['tmpStatusExterno'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_StatusExt', 'StatusExterno']].drop_duplicates()
    dfx.rename(columns={'FK_dim_StatusExt': 'PK_dim_StatusExt'}, inplace = True)

    # exclui a coluna Grupo de df_tasks que será a futura tabela fato
    df.drop(['StatusExterno', 'tmpStatusExterno'], axis=1, inplace=True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['StatusExterno'] == '', ['StatusExterno']] = '<sem status>'

    print('dim_StatusExt montada')
    
    return dfx

## DIM usuario

In [16]:
def Mt_Usuarios(df):


    # trasnforma a coluna Usuario em uma coluna do tipo "category'
    df['tmpUsuario'] = df['Usuarios'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Usuario
    df['FK_dim_Usuarios'] = df['tmpUsuario'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Usuarios', 'Usuarios']].drop_duplicates()
    col_ren = {'FK_dim_Usuarios': 'PK_dim_Usuarios', 'Usuarios': 'Usuario'}
    dfx.rename(columns=col_ren, inplace = True)

    # exclui a coluna Usuario de df_tasks que será a futura tabela fato
    df.drop(['Usuarios', 'tmpUsuario'], axis=1, inplace=True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Usuario'] == '', ['Usuario']] = '<usuário indefinido>'

    print('dim_Usuario montada')
    
    return dfx

## DIM situacao

In [17]:
def Mt_Situacao():

    dfx = pd.DataFrame({'PK_dim_Situacao': [1, 2, 3],
                        'Situacao':        ['em Andamento', 'Encerrada', 'Cancelada']}
                        )
    print('dim_Situacao montada')

    return dfx


## DIM SLA

In [18]:
def Mt_SLA():

    dfx = pd.DataFrame({'PK_dim_SLA': [1, 2, 3],
                        'DescSLA':    ['dentro do prazo', 'perto do prazo', 'fora do prazo'],
                        'Cor':        ['verde','amarelo' ,'vermelho']}
                    )
    print('dim_SLA montada')

    return dfx

## DIM endereco

In [19]:
def Mt_Endereco(df):

    dict_uf = {'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia',
               'CE': 'Ceará', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão', 'MT': 'Mato Grosso',
               'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
               'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro',
               'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima',
               'SC': 'Santa Catarina', 'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins', 'DF': 'Distrito Federal',
               '<nd>': '<sem UF>'
    }

    # acrescenta o nome do estado ao dataframe
    dfx = df.copy()
    dfx['NomeUF'] = dfx['UF'].apply(lambda x: dict_uf.get(x, None))
    dfx.rename(columns={'Solicitacao': 'PK_dim_Endereco'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['CEP'].isna(), ['CEP']] = ''

    print('dim_Endereco montada')

    return dfx

## FAT tasks

In [20]:
# função retorna a situação da solicitação de acordo com as coilunas cancelado e encerrado
def RetSit(Enc, Canc):
    rt = 0
    if Canc == 1:
        rt = 3
    elif Enc == 1:
        rt = 2
    else:
        rt = 1
    return rt

In [21]:
def Mt_Tasks(df, dfr):

    dfx = df.copy()
    
    # faz um merge com o dataset de ratings
    dfx = df.merge(dfr, left_on='Protocolo', right_on='Solicitacao', how='left')
    dfx.drop(['Solicitacao'], axis=1, inplace=True)

    # define o código de situação da solicitação
    df_aux = dfx[['Protocolo', 'ProcessoEncerrado', 'ProcessoCancelado']].drop_duplicates()

    # substui valores de colunas
    troca = {'false': 0, 'true': 1}
    df_aux['ProcessoEncerrado'] = df_aux['ProcessoEncerrado'].map(troca)
    df_aux['ProcessoCancelado'] = df_aux['ProcessoCancelado'].map(troca)

    # agrega as ações para um registro por protocolo
    df_aux = df_aux.groupby('Protocolo').agg({'ProcessoEncerrado': 'max', 'ProcessoCancelado': 'max'}).reset_index()

    # retorna a situção da solicitação: 1-em andamento, 2-encerrado, 3-cancelado
    df_aux['Situacao'] = df_aux.apply(lambda x: RetSit(x['ProcessoEncerrado'], x['ProcessoCancelado']), axis=1)
    df_aux.drop(['ProcessoEncerrado', 'ProcessoCancelado'], axis=1, inplace=True)

    # faz merge com o dataset de situação da solicitação
    dfx = dfx.merge(df_aux, left_on='Protocolo', right_on='Protocolo', how='left')
    dfx.drop(['ProcessoEncerrado', 'ProcessoCancelado'], axis=1, inplace=True)

    # coloca -1 para as solicitações que não possuem avaliação
    dfx.loc[dfx['NotaAvaliacao'].isna(), 'NotaAvaliacao'] = -1
    dfx['NotaAvaliacao'] = dfx['NotaAvaliacao'].astype('int32')

    # renomeia as colunas
    col_ren = {'Protocolo': 'FK_dim_Solicitacoes', 'Situacao': 'FK_dim_Situacao'}
    dfx.rename(columns=col_ren, inplace = True)

    # trata a coluna Prazo
    dfx['Prazo'] == ''
    dfx.loc[dfx['Prazo'] == '', 'Prazo'] = '0'
    dfx['Prazo'] = dfx['Prazo'].astype('int64')

    dfx['DataCriacao'] = pd.to_datetime(dfx['DataHora_Criacao']).dt.date
    dfx['HoraCriacao'] = pd.to_datetime(dfx['DataHora_Criacao']).dt.time

    dfx['DataConclusao'] = pd.to_datetime(dfx['DataHora_Conclusao']).dt.date
    dfx['HoraConclusao'] = pd.to_datetime(dfx['DataHora_Conclusao']).dt.time

    dfx.drop(['DataHora_Criacao', 'DataHora_Conclusao'], axis=1, inplace=True)

    cols_ordem = ['FK_dim_Solicitacoes', 'Prazo', 'DataCriacao', 'HoraCriacao', 
                  'DataConclusao', 'HoraConclusao', 
                  'NotaAvaliacao', 'MotivoAvaliacao', 
                  'DataAvaliacao', 'HoraAvaliacao', 
                  'FK_dim_Entidades', 'FK_dim_Servicos', 'FK_dim_Usuarios', 'FK_dim_GruposUsuarios', 
                  'FK_dim_Acoes', 'FK_dim_StatusExt', 'FK_dim_CategoriasServicos', 'FK_dim_GrupoResponsavel', 
                  'FK_dim_MotivosCanc', 'FK_dim_Encaminhamento', 'FK_dim_Situacao'
                ]
    dfx = dfx[cols_ordem]

    print('fat_Tasks montada')

    return dfx


# <font color='red'>4.0 LOAD</font>

In [22]:
def Load_BD(strAut):

    # import the module
    # create sqlalchemy engine
    
    SQLengine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                           .format(user = strAut['My_user'],
                                   pw   = strAut['My_pw'],
                                   host = strAut['My_host'],
                                   db   = strAut['My_db']),
                                   pool_recycle=3600)
    dbConn = SQLengine.connect()

    try:
        df_dim_Acao.to_sql('dim_acoes', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Categoria.to_sql('dim_categorias_servicos', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Encaminhamento.to_sql('dim_encaminhamento', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Entidade.to_sql('dim_entidades', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_GrupoResponsavel.to_sql('dim_grupo_responsavel', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Grupo.to_sql('dim_grupos_usuarios', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_MotivoCanc.to_sql('dim_motivos_canc', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Servicos.to_sql('dim_servicos', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_StatusExt.to_sql('dim_status_ext', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Usuarios.to_sql('dim_usuarios', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Situacao.to_sql('dim_situacao', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_SLA.to_sql('dim_sla', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Endereco.to_sql('dim_endereco', con=dbConn, if_exists='replace', index=False, chunksize = 1000)

        df_fat_tasks.to_sql('fat_tasks', con=dbConn, if_exists='replace', index=False, chunksize = 1000)

    except ValueError as vx:
        print('ERROR -', vx)

    except Exception as ex:
        print('EXCEPTION -', ex)

    else:
        print('Tabelas criadas com sucesso');  

    finally:
        dbConn.close()

# <font color='red'>0.0 MAIN</font>

In [23]:
# monta as strings de conexão
def StrConn(sit):
    in_oper   = {'My_host': 'localhost', 'My_db': 'bd_fontes',       'My_user': 'gd', 'My_pw': 'Alpar@123'}
    out_oper  = {'My_host': 'localhost', 'My_db': 'bd_dw',           'My_user': 'gd', 'My_pw': 'Alpar@123'}
    in_teste  = {'My_host': 'localhost', 'My_db': 'bd_teste_fontes', 'My_user': 'gd', 'My_pw': 'Alpar@123'}
    out_teste = {'My_host': 'localhost', 'My_db': 'bd_teste_dw',     'My_user': 'gd', 'My_pw': 'Alpar@123'}
    
    # se True é teste
    if sit:
        rt_in = in_teste
        rt_out = out_teste
    else:
        rt_in = in_oper
        rt_out = out_oper

    return rt_in, rt_out

In [24]:
if __name__ == '__main__':
    
    etl_teste = False

    # EXTRACT
    AUTENTIC_IN, AUTENTIC_OUT = StrConn(etl_teste)
    df_tasks, df_sla, df_form, df_rating = LeFontes(AUTENTIC_IN)
    
    # TRANSFORM
    # trata os datasets
    df_sla    = trSLA(df_sla)
    df_form   = trForm(df_form)
    df_rating = trRating(df_rating)
    df_tasks  = trTasks(df_tasks)
    
    # monta as tabelas dimensão
    df_dim_Acao              = Mt_dim_Acoes(df_tasks)
    df_dim_Categoria         = Mt_dim_CategoriaServico(df_tasks)
    df_dim_Encaminhamento    = Mt_dim_Encaminhamento(df_tasks)
    df_dim_Entidade          = Mt_dim_Entidade(df_tasks)
    df_dim_GrupoResponsavel  = Mt_dim_GrupoResponsavel(df_tasks)
    df_dim_Grupo             = Mt_dim_GruposUsuarios(df_tasks)
    df_dim_MotivoCanc        = Mt_MotivosCanc(df_tasks)
    df_dim_Servicos          = Mt_Servico(df_tasks, df_sla)
    df_dim_StatusExt         = Mt_StatusExt(df_tasks)
    df_dim_Usuarios          = Mt_Usuarios(df_tasks)
    df_dim_Situacao          = Mt_Situacao()
    df_dim_SLA               = Mt_SLA()
    df_dim_Endereco          = Mt_Endereco(df_form)
    
    # monta a tabela fato
    df_fat_tasks             = Mt_Tasks(df_tasks, df_rating)
    
    # monta as tabelas dimensão do META BI
    
    
    # LOAD
    ret = Load_BD(AUTENTIC_OUT)
    

tasks:619 registros lidos em 16 colunas
sla: 54 registros lidos em 5 colunas
form: 750 registros lidos em 3 colunas
rating: 30 registros lidos em 4 colunas
MySQL connection is closed
df_sla transformado
df_form transformado
df_rating transformado
df_tasks transformado
dim_Acoes montada
dim_Categoria montada
dim_Encaminhamento montada
dim_Entidade montada
dim_GrupoResponsavel montada
dim_GruposUsuarios montada
dim_MotivosCanc montada
dim_Servico montada
dim_StatusExt montada
dim_Usuario montada
dim_Situacao montada
dim_SLA montada
dim_Endereco montada
fat_Tasks montada
Tabelas criadas com sucesso
